# 호텔 소개를 통한 추천 시스템

In [ ]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 448 kB 38.3 MB/s 
     |████████████████████████████████| 86 kB 5.5 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
import pandas as pd
import re
from konlpy.tag import Okt

In [ ]:
df = pd.read_excel('info.xlsx')
df

,Unnamed: 0,ID,region,name,star,rating,num_review,basic_info,keywords
0,0,3001626,0,크라운파크호텔 서울,3,4.6,48,비즈니스 관광 문화의 중심지 명동에 위치한 크라운 파크 호텔은 서울의 역사와...,"['비즈니스', '관광', '문화', '중심지', '명동', '위치', '크라운',..."
1,1,3000741,0,써미트 호텔 동대문,3,4.5,209,깨끗하고 쾌적했다는 투숙객들의 이용후기가 줄을 잇네요 오래된 특1급 호텔...,"['깨끗하고', '쾌적했다는', '투숙', '이용', '후기', '잇네요', '오래..."
2,2,1000112780,0,어반스테이 더 남산,0,4.8,241,어반스테이 더 남산은 비대면 체크인 으로 안전하게 머물고 삶이 좀 더 가벼워...,"['스테이', '남산', '비대', '체크', '으로', '안전하게', '머물고',..."
3,3,3014025,0,영등포 갤럭시 호텔,0,4.1,370,갤럭시 관광호텔은 서울시 영등포구 버드 나루로 90번지 당산동 에 위치하여 인천...,"['갤럭시', '관광', '호텔', '서울시', '영등포구', '버드', '나루',..."
4,4,3000607,0,호텔 더 디자이너스 홍대,0,3.7,803,각종 잡지 및 화보 촬영에도 사용되는 아름다운 객실에서 즐기는 특별한 휴식을 즐...,"['각종', '잡지', '화보', '촬영', '에도', '사용', '되는', '아름..."
...,...,...,...,...,...,...,...,...,...
2101,130,3000818,8,라미아 레지던스,0,4.4,248,2011년 봄 신축 개업하여 깨끗한 객실 상태를 유지하고 있는 레지던스형 호텔 ...,"['신축', '개업', '하여', '깨끗한', '객실', '상태', '유지', '하..."
2102,131,3000789,8,호텔 머드린,0,4.4,445,세계적인 관광지 대천해수욕장까지 도보로 1분거리 보령 머드축제가 호텔 바...,"['세계', '관광지', '대천해수욕장', '까지', '도보', '거리', '보령'..."
2103,132,3000753,8,베니키아 테크노밸리호텔,0,4.6,455,2013년 12월 그랜드 오픈한 호텔입니다 다양한 체험과학시설 및 계룡산 국...,"['그랜드', '오픈', '호텔', '입니다', '다양한', '체험', '과학', ..."
2104,133,3000738,8,굿모닝 레지던스호텔 휴,0,4.7,452,대전지역 거의 유일한 레지던스 호텔 심플하고 깨끗해요 아메리칸 스타일의 조식...,"['대전', '지역', '거의', '유일한', '레지던스', '호텔', '심플', ..."


# 호텔 소개 데이터 전처리

In [ ]:
"""한글, 숫자, 영어 빼고 전부 제거"""
def sub_special(s):
  return re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣0-9a-zA-Z ]','',s)

In [ ]:
df['basic_info'] = df['basic_info'].apply(sub_special)

In [ ]:
df

,Unnamed: 0,ID,region,name,star,rating,num_review,basic_info,keywords
0,0,3001626,0,크라운파크호텔 서울,3,4.6,48,비즈니스 관광 문화의 중심지 명동에 위치한 크라운 파크 호텔은 서울의 역사와...,"['비즈니스', '관광', '문화', '중심지', '명동', '위치', '크라운',..."
1,1,3000741,0,써미트 호텔 동대문,3,4.5,209,깨끗하고 쾌적했다는 투숙객들의 이용후기가 줄을 잇네요 오래된 특1급 호텔...,"['깨끗하고', '쾌적했다는', '투숙', '이용', '후기', '잇네요', '오래..."
2,2,1000112780,0,어반스테이 더 남산,0,4.8,241,어반스테이 더 남산은 비대면 체크인 으로 안전하게 머물고 삶이 좀 더 가벼워...,"['스테이', '남산', '비대', '체크', '으로', '안전하게', '머물고',..."
3,3,3014025,0,영등포 갤럭시 호텔,0,4.1,370,갤럭시 관광호텔은 서울시 영등포구 버드 나루로 90번지 당산동 에 위치하여 인천...,"['갤럭시', '관광', '호텔', '서울시', '영등포구', '버드', '나루',..."
4,4,3000607,0,호텔 더 디자이너스 홍대,0,3.7,803,각종 잡지 및 화보 촬영에도 사용되는 아름다운 객실에서 즐기는 특별한 휴식을 즐...,"['각종', '잡지', '화보', '촬영', '에도', '사용', '되는', '아름..."
...,...,...,...,...,...,...,...,...,...
2101,130,3000818,8,라미아 레지던스,0,4.4,248,2011년 봄 신축 개업하여 깨끗한 객실 상태를 유지하고 있는 레지던스형 호텔 ...,"['신축', '개업', '하여', '깨끗한', '객실', '상태', '유지', '하..."
2102,131,3000789,8,호텔 머드린,0,4.4,445,세계적인 관광지 대천해수욕장까지 도보로 1분거리 보령 머드축제가 호텔 바...,"['세계', '관광지', '대천해수욕장', '까지', '도보', '거리', '보령'..."
2103,132,3000753,8,베니키아 테크노밸리호텔,0,4.6,455,2013년 12월 그랜드 오픈한 호텔입니다 다양한 체험과학시설 및 계룡산 국...,"['그랜드', '오픈', '호텔', '입니다', '다양한', '체험', '과학', ..."
2104,133,3000738,8,굿모닝 레지던스호텔 휴,0,4.7,452,대전지역 거의 유일한 레지던스 호텔 심플하고 깨끗해요 아메리칸 스타일의 조식...,"['대전', '지역', '거의', '유일한', '레지던스', '호텔', '심플', ..."


In [ ]:
df['keywords']

0       ['비즈니스', '관광', '문화', '중심지', '명동', '위치', '크라운',...
1       ['깨끗하고', '쾌적했다는', '투숙', '이용', '후기', '잇네요', '오래...
2       ['스테이', '남산', '비대', '체크', '으로', '안전하게', '머물고',...
3       ['갤럭시', '관광', '호텔', '서울시', '영등포구', '버드', '나루',...
4       ['각종', '잡지', '화보', '촬영', '에도', '사용', '되는', '아름...
                              ...                        
2101    ['신축', '개업', '하여', '깨끗한', '객실', '상태', '유지', '하...
2102    ['세계', '관광지', '대천해수욕장', '까지', '도보', '거리', '보령'...
2103    ['그랜드', '오픈', '호텔', '입니다', '다양한', '체험', '과학', ...
2104    ['대전', '지역', '거의', '유일한', '레지던스', '호텔', '심플', ...
2105    ['거리', '두기', '단계', '일상', '회복', '개편', '방역', '행정...
Name: keywords, Length: 2106, dtype: object

# TF-IDF 생성

In [ ]:
"""TF-IDF로 만들기"""
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df.basic_info)

# 호텔 소개에 대해서 tf-idf 수행
print(tfidf_matrix.shape) # -> 데이터개수: 2106, 유니크한 단어들 개수: 24442개

(2106, 24442)


# 코사인 유사도 구하기
-> 1에 가까울수록 유사한 호텔임

In [ ]:
"""코사인 유사도 구하기"""
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
"""인덱스 테이블 만들기"""
indices = pd.Series(df.index, index=df.name).drop_duplicates()
print(indices)

name
크라운파크호텔 서울          0
써미트 호텔 동대문          1
어반스테이 더 남산          2
영등포 갤럭시 호텔          3
호텔 더 디자이너스 홍대       4
                 ... 
라미아 레지던스         2101
호텔 머드린           2102
베니키아 테크노밸리호텔     2103
굿모닝 레지던스호텔 휴     2104
유성호텔             2105
Length: 2106, dtype: int64


In [ ]:
"""추천 해주기"""
def hotel_REC(name, cosine_sim=cosine_sim):
    #입력한 영화로 부터 인덱스 가져오기
    idx = indices[name]

    # 모든 영화에 대해서 해당 영화와의 유사도를 구하기
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 영화들을 정렬
    sim_scores = sorted(sim_scores, key=lambda x:x[1], reverse = True)

    # 가장 유사한 10개의 영화를 받아옴
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개 영화의 인덱스 받아옴
    hotel_indices = [i[0] for i in sim_scores]
    
    #기존에 읽어들인 데이터에서 해당 인덱스의 값들을 가져온다. 그리고 스코어 열을 추가하여 코사인 유사도도 확인할 수 있게 한다.
    result_df = df.iloc[hotel_indices].copy()
    result_df['score'] = [i[1] for i in sim_scores]
    
    # 읽어들인 데이터에서 줄거리 부분만 제거, 제목과 스코어만 보이게 함
    result_df = result_df.drop(df.columns[[0, 1, 7, 8]], axis=1)

    # 가장 유사한 10개의 영화의 제목을 리턴
    return result_df

In [ ]:
hotel_REC("노보텔 앰배서더 동대문 호텔 & 레지던스")

,region,name,star,rating,num_review,score
103,0,노보텔 스위트 앰배서더 서울 용산,5,4.8,943,0.140398
98,0,노보텔 앰배서더 서울 용산,5,4.7,4043,0.119759
95,0,인터컨티넨탈 서울 코엑스,5,4.8,1023,0.093220
117,0,이비스 스타일 앰배서더 서울 용산,4,4.6,2389,0.092238
114,0,힐튼 가든인 서울 강남,0,4.8,78,0.089754
439,1,페어필드 바이 메리어트 부산,0,4.7,500,0.087089
1716,6,대구 아리아나 호텔,3,4.5,150,0.086422
78,0,나인트리 호텔 동대문,0,4.8,2209,0.086398
845,2,베이힐 풀앤빌라,0,4.8,22,0.083342
177,0,코트야드 메리어트 서울 남대문,4,4.7,262,0.081794
